WOA Clustering Algorithm

In [34]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from numpy.linalg import norm

In [35]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data',header=None)
scaler = MinMaxScaler()
df = scaler.fit_transform(df)
df = pd.DataFrame(df)
df = df.sample(frac=1)

In [36]:
df.columns = ["class",
            "alcohol",
            "malic_acid",
            "ash",
            "alcalinity_of_ash",
            "magnesium",
            "total_phenols",
            "flavanoids",
            "nonflavanoid_phenols",
            "proanthocayanins",
            "color_intensity",
            "hue",
            "od280",
            "proline"]
df = df.drop(["class"],axis=1)

In [37]:
def init_whale(spaceDim,numCenters,numPopulation,lowerBound,upperBound):
    whalePop = []
    for i in range(numPopulation):
        tmp = np.random.rand(numCenters*spaceDim)*(upperBound-lowerBound)-(upperBound-lowerBound)/2
        whalePop.append(tmp)
    return whalePop

In [38]:
maxIter = 100
numPopulation = 10
spaceDim = len(df.columns)
P = len(df) #178
numCenters = 3
lowerBound = 0 #must be negativ
upperBound = 1 #must be positive
b = 1.0 #dd0.618
#LSSI = 999999999

In [39]:
whalePop = init_whale(spaceDim,numCenters,numPopulation,lowerBound,upperBound)

In [40]:
t = 0
count = 0
bestWhale = None
currentBestWhale = None
fitness_vec = []

while t<maxIter:        
    
    #STEP 1 ---------------------------------------
    
    fitnessPop = []
    for whale in whalePop:

        fitnessWhale = 0
        for p in range(P):
            distances = [norm(df.iloc[p]-whale[0:13]),norm(df.iloc[p]-whale[13:26]),norm(df.iloc[p]-whale[26:39])]
            idxDist = np.argmin(distances)
            fitnessWhale += distances[idxDist]
        fitnessPop.append(fitnessWhale)
        count += 1
    idxFit = np.argmin(fitnessPop)
    bestWhale = whalePop[idxFit]
    fitness_vec.append(fitnessPop[idxFit])
    
    print("Iteration "+str(t)+", Fitness of best whale "+str(fitnessPop[idxFit]))
        
    #STEP 2 ---------------------------------------
    
    for i,whale in enumerate(whalePop,0):
        
        if i != idxFit: #keep best whale
            
            #Update values a,A,C,l,p
            a = 2-(2*t)/maxIter #decreases linearly from 2 to 0
            r = np.ones(numCenters*spaceDim)*np.random.uniform(0,1,1)[0] #a vector filled with the same random number in [0,1]
            A = 2*a*r-a #a vector filled with the same random value in [-a,a]
            C = 2*r
            p = np.random.uniform(0,1,1)[0]
            l = np.random.uniform(-1,1,1)[0]

            if p < 0.5:
                if norm(A) < 1:

                    #Phase of encircling
                    #Note that with increasing iteration exploration will stop
                    #D = np.abs(np.multiply(C,bestWhale)-whale)
                    D = np.multiply(C,bestWhale)-whale
                    whalePop[i] = bestWhale-np.multiply(A,D)

                elif norm(A) >= 1: 

                    #Phase of exploration
                    XRand = np.random.rand(numCenters*spaceDim) #must not necessarily be in [0,1]
                    #D = np.abs(np.multiply(C,bestWhale)-whale)
                    D = np.multiply(C,bestWhale)-whale
                    whalePop[i] = XRand-np.multiply(A,D)

            elif p>= 0.5:

                #Phase of spiral
                D_ = bestWhale-whale
                whalePop[i] = D_*np.exp(b*l)*np.cos(2*np.pi*l)+bestWhale

            #Bring back whale which is out of search space
            for j,coordinate in enumerate(bestWhale,0):
                if coordinate > 1:
                    bestWhale[j] = 0.5
                elif coordinate < 0:
                    bestWhale[j] = 0.5
            
    t += 1

Iteration 0, Fitness of best whale 269.0028008831346
Iteration 1, Fitness of best whale 164.48050452194008
Iteration 2, Fitness of best whale 156.91849888890104
Iteration 3, Fitness of best whale 153.4343080979847
Iteration 4, Fitness of best whale 148.01773973430338
Iteration 5, Fitness of best whale 147.24336828602455
Iteration 6, Fitness of best whale 147.15476275212012
Iteration 7, Fitness of best whale 142.04539210169688
Iteration 8, Fitness of best whale 140.3765728291574
Iteration 9, Fitness of best whale 139.45443405820433
Iteration 10, Fitness of best whale 137.001440581995
Iteration 11, Fitness of best whale 131.39942948951798
Iteration 12, Fitness of best whale 130.24182387046866
Iteration 13, Fitness of best whale 128.61634648180188
Iteration 14, Fitness of best whale 128.61634648180188
Iteration 15, Fitness of best whale 128.61634648180188
Iteration 16, Fitness of best whale 128.39719242306592
Iteration 17, Fitness of best whale 128.39719242306592
Iteration 18, Fitness of 

In [41]:
z1,z2,z3 = bestWhale[0:13],bestWhale[13:26],bestWhale[26:39]

In [42]:
cluster = []
for p in range(P):
    distances = [norm(df.iloc[p]-z1),norm(df.iloc[p]-z2),norm(df.iloc[p]-z3)]
    cluster.append(np.argmin(distances)+1)
df["cluster"] = cluster

In [43]:
SSE = 0
df_tmp = df.drop(["cluster"],axis=1)
for p in range(P):
    if df.iloc[p].cluster == 1:
        SSE += norm(df_tmp.iloc[p]-z1)
    if df.iloc[p].cluster == 2:
        SSE += norm(df_tmp.iloc[p]-z2)
    if df.iloc[p].cluster == 3:
        SSE += norm(df_tmp.iloc[p]-z3)
print("SSE: "+str(SSE/len(df))) #best:0.59

SSE: 0.6131658721339163


In [44]:
x = np.sort(df.cluster.values)
count1 = 0
count2 = 0
count3 = 0
for i in range(len(x)):
    if x[i] == 1:
        count1 += 1
    if x[i] == 2:
        count2 += 1
    if x[i] == 3:
        count3 += 1
print(count1,count2,count3) #should be (59,71,48)

59 57 62
